In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing

In [23]:
def processar_dados_ano(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo, encoding='latin1', delimiter=';')

    colunas_para_remover = [col for col in df.columns if (col.startswith('DS_') or col.startswith('NU_') or col.startswith('DT_') or col.startswith('CO_') or col.startswith('NO_')) and col != 'NU_ANO_CENSO']
    df_limpo = df.drop(columns=colunas_para_remover)

    colunas_com_nan = df_limpo.columns[df_limpo.isnull().any()].tolist()

    return df_limpo, colunas_com_nan

dfs = list()
diretorio = 'dataset/'
anos = range(2014, 2022)  

for ano in anos:
    caminho_arquivo = f'{diretorio}microdados_ed_basica_{ano}_alagoas.csv'
    df_limpo, colunas_com_nan = processar_dados_ano(caminho_arquivo)
    dfs.append(df_limpo)
    print(f"Resumo dos dados de {ano}:")
    print(colunas_com_nan)
    print("--------------------------------------------------------------------\n")
    print(df_limpo.describe())

Resumo dos dados de 2014:
[]
--------------------------------------------------------------------

       NU_ANO_CENSO  TP_DEPENDENCIA  TP_CATEGORIA_ESCOLA_PRIVADA  \
count        3787.0     3787.000000                  3787.000000   
mean         2014.0        3.053868                     0.178241   
std             0.0        0.495892                     0.547454   
min          2014.0        1.000000                     0.000000   
25%          2014.0        3.000000                     0.000000   
50%          2014.0        3.000000                     0.000000   
75%          2014.0        3.000000                     0.000000   
max          2014.0        4.000000                     4.000000   

       TP_LOCALIZACAO  TP_LOCALIZACAO_DIFERENCIADA  TP_SITUACAO_FUNCIONAMENTO  \
count     3787.000000                  3787.000000                3787.000000   
mean         1.549247                     0.071032                   1.155004   
std          0.497634                     0.4

In [36]:
df_consolidado = pd.concat(dfs)